In [1]:
import pandas as pd

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/keremgura/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/keremgura/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/keremgura/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/keremgura/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/keremgura/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/keremgura/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/keremgura/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/keremgura/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.st

AttributeError: _ARRAY_API not found

In [2]:
DATA_PATH = '../../data/'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
SKILLCORNER_RAW_PATH = DATA_PATH + 'skillcorner_raw/'
XT_PLOT_PATH = DATA_PATH + 'smoothed_xt.csv'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'
WYSCOUT_TO_SKILLCORNER = DATA_PATH + 'wyscout2skillcorner.csv'

## Load Data

In [3]:
match_info = pd.read_csv(MATCH_IDS_PATH)
match_info = match_info.dropna(subset=['wyscout', 'skillcorner'])
match_info['skillcorner'] = match_info['skillcorner'].astype(int)

In [4]:
match_info.head(5)

,wyscout,skillcorner,date,home,result,away
8,5414111,952209,2023-03-24,France,4-0,Netherlands
10,5414113,726849,2023-03-24,Czech Republic,3-1,Poland
12,5414115,722109,2023-03-24,Sweden,0-3,Belgium
25,5414127,726848,2023-03-26,England,2-0,Ukraine
33,5414135,952208,2023-03-27,Republic of Ireland,0-1,France


In [5]:
wyscout_id = match_info['wyscout'].values[0]
skillcorner_id = match_info['skillcorner'].values[0]

### wyscout

In [6]:
wyscout_data = wyscout_to_df(WYSCOUT_PATH + str(wyscout_id) + ".json")
with pd.option_context('display.max_columns', None):
    display(wyscout_data.head(3))

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
0,1587492614,5414111,1H,0,1,00:00:01.912,3.912638,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",51,50,664,Netherlands,4-2-3-1,4418,France,4-3-3,118,M. Depay,CF,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,1.587493e+09,110.275779,[],43.0,0.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1587492615,5414111,1H,0,5,00:00:05.782,7.782226,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",42,83,664,Netherlands,4-2-3-1,4418,France,4-3-3,625,M. de Roon,RDMF,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,1.587493e+09,110.275779,[],43.0,1.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1587492616,5414111,1H,0,8,00:00:08.951,10.951834,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",29,60,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,1.587493e+09,110.275779,[],43.0,2.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### skillcorner

In [7]:
metadata = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_metadata.csv")
tracking_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_tracking.csv")
lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")
play_direction_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_play_direction.csv")

In [8]:
print("Metadata")
display(metadata.head(3))
print("Tracking")
display(tracking_df.head(3))
print("Lineup")
display(lineup_df.head(3))
print("Play Direction")
display(play_direction_df)


Metadata


,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,home_team_jersey_color,away_team_jersey_color,home_team_number_color,away_team_number_color,home_team_coach,away_team_coach,pitch_name,pitch_length,pitch_width,provider,fps
0,952209,03/24/2023 19:45,European Championship Qualifiers,2023/2024,France,Netherland,4,0,#fafafa,#ff824c,#000000,#000000,Didier Deschamps,Frank De Boer,Stade de France,105.0,68.0,SkillCorner,10


Tracking


,match_id,half,frame_id,timestamp,object_id,x,y,z,extrapolated
0,952209,1,20,0,-1,0.12,-0.19,0.12,False
1,952209,1,20,0,7604,-39.44,-0.21,0.00,True
2,952209,1,20,0,16320,-16.05,-7.18,0.00,False


Lineup


,match_id,team_name,player_id,player_first_name,player_last_name,player_shirt_number,player_position,player_birthdate,start_time,end_time,yellow_card,red_card,injured,goal,own_goal
0,952209,Netherland,14397,Tyrell,Malacia,16,Right Wing Back,1999-08-17,01:26:59,NaN,0,0,False,0,0
1,952209,Netherland,12654,Donyell,Malen,7,Right Winger,1999-01-19,01:06:22,NaN,0,0,False,0,0
2,952209,Netherland,2548,Davy,Klaassen,14,Attacking Midfield,1993-02-21,01:06:22,NaN,0,0,False,0,0


Play Direction


,match_id,team_name,half,play_direction
0,952209,France,1,BOTTOM_TO_TOP
1,952209,Netherland,1,TOP_TO_BOTTOM
2,952209,France,2,TOP_TO_BOTTOM
3,952209,Netherland,2,BOTTOM_TO_TOP


# Pass Events

In [9]:
passes_df = wyscout_data[wyscout_data['type.primary'] == 'pass']

# Filter out inaccurate passes
accurate_only = True
if accurate_only:
    passes_df = passes_df[passes_df['pass.accurate'] == True]
    
# Filter out passes with invalid recipient id
passes_df = passes_df[passes_df['pass.recipient.id'] != 0]

print(passes_df.shape)

(894, 93)


In [13]:
pitch_length = int(metadata.iloc[0]['pitch_length'])
pitch_width = int(metadata.iloc[0]['pitch_width'])

In [14]:
passes = passes_df.loc[:,['matchTimestamp', 'matchPeriod', 'player.id', 'pass.recipient.id', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']]
passes.head()

,matchTimestamp,matchPeriod,player.id,pass.recipient.id,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1H,118,625.0,51,50,42.0,83.0
1,00:00:05.782,1H,625,476361.0,42,83,29.0,60.0
2,00:00:08.951,1H,476361,370.0,29,60,25.0,28.0
3,00:00:13.151,1H,370,476361.0,25,28,25.0,51.0
5,00:00:17.930,1H,476361,370.0,32,50,30.0,17.0


In [15]:
passes['matchPeriod'] = passes['matchPeriod'].apply(lambda x: int(x.split('H')[0]))
passes['pass.recipient.id'] = passes['pass.recipient.id'].astype(int)
passes['pass.endLocation.x'] = passes['pass.endLocation.x'].astype(int)
passes['pass.endLocation.y'] = passes['pass.endLocation.y'].astype(int)

In [16]:
passes.head()

,matchTimestamp,matchPeriod,player.id,pass.recipient.id,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1,118,625,51,50,42,83
1,00:00:05.782,1,625,476361,42,83,29,60
2,00:00:08.951,1,476361,370,29,60,25,28
3,00:00:13.151,1,370,476361,25,28,25,51
5,00:00:17.930,1,476361,370,32,50,30,17


# Map Wyscout IDs to SkillCorner IDs

In [17]:
wyscout2skillcorner = pd.read_csv(WYSCOUT_TO_SKILLCORNER).drop(columns='id')

In [18]:
passes.rename(columns={'player.id': 'player.id.wyscout', 'pass.recipient.id': 'pass.recipient.id.wyscout'}, inplace=True)

In [19]:
passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy']], left_on='player.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'player.id.skillcorner'}, inplace=True)

passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy']], left_on='pass.recipient.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'pass.recipient.id.skillcorner'}, inplace=True)

In [20]:
cols = ['matchTimestamp', 'matchPeriod', 'player.id.wyscout', 'player.id.skillcorner', 'pass.recipient.id.wyscout', 'pass.recipient.id.skillcorner', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']
passes = passes.loc[:,cols]

In [21]:
passes.head()

,matchTimestamp,matchPeriod,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1,118,7387,625,7017,51,50,42,83
1,00:00:05.782,1,625,7017,476361,14407,42,83,29,60
2,00:00:08.951,1,476361,14407,370,10738,29,60,25,28
3,00:00:13.151,1,370,10738,476361,14407,25,28,25,51
4,00:00:17.930,1,476361,14407,370,10738,32,50,30,17


# Lineup Information

# Play Direction

In [22]:
play_direction_dict = play_direction_df.drop(columns='match_id').set_index(['team_name', 'half']).to_dict()['play_direction']
play_direction_dict

{('France', 1): 'BOTTOM_TO_TOP',
 ('Netherland', 1): 'TOP_TO_BOTTOM',
 ('France', 2): 'TOP_TO_BOTTOM',
 ('Netherland', 2): 'BOTTOM_TO_TOP'}

In [23]:
player_team_dict = lineup_df[['player_id', 'team_name']].set_index('player_id').to_dict()['team_name']

In [24]:
def get_play_direction(row):
    team_name = player_team_dict[row['player.id.skillcorner']]
    return play_direction_dict[(team_name, row['matchPeriod'])]

In [25]:
passes['play_direction'] = passes.apply(get_play_direction, axis=1)

In [27]:
# read xt table
xt_table = pd.read_csv(XT_PLOT_PATH)

cell_width = 100 / pitch_length
cell_height = 100 / pitch_width

print(f'cell width:\t{round(cell_width,2)}')
print(f'cell height:\t{round(cell_height,2)}')

cell width:	0.95
cell height:	1.47


In [28]:
def get_xt_index(x, y, direction):
    # Adjust x-coordinate if the direction is left (-1)
    x_adjusted = x if direction == 1 else pitch_length - x
    y_adjusted = y if direction == 1 else pitch_width - y

    x_index = min(int(x_adjusted // cell_width), pitch_length - 1)
    y_index = min(int(y_adjusted // cell_height), pitch_width - 1)
    return x_index, y_index

passes['start_xt_right'] = passes.apply(lambda row: xt_table.iat[get_xt_index(row['location.x'], row['location.y'], 1)[1], 
                                                                 get_xt_index(row['location.x'], row['location.y'], 1)[0]], axis=1)
passes['end_xt_right'] = passes.apply(lambda row: xt_table.iat[get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'], 1)[1], 
                                                               get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'], 1)[0]], axis=1)

# Calculate xT values for attacking left
passes['start_xt_left'] = passes.apply(lambda row: xt_table.iat[get_xt_index(row['location.x'], row['location.y'], -1)[1], 
                                                                get_xt_index(row['location.x'], row['location.y'], -1)[0]], axis=1)
passes['end_xt_left'] = passes.apply(lambda row: xt_table.iat[get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'], -1)[1], 
                                                              get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'], -1)[0]], axis=1)

# Calculate dxt for both directions
passes['dxt_right'] = passes['end_xt_right'] - passes['start_xt_right']
passes['dxt_left'] = passes['end_xt_left'] - passes['start_xt_left']

# Add a new column 'dxt' based on the play_direction condition
passes['dxt'] = passes.apply(lambda row: row['dxt_left'] if row['play_direction'] == 'TOP_TO_BOTTOM' else row['dxt_right'], axis=1)

In [29]:
passes

,matchTimestamp,matchPeriod,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,start_xt_right,end_xt_right,start_xt_left,end_xt_left,dxt_right,dxt_left,dxt
0,00:00:01.912,1,118,7387,625,7017,51,50,42,83,TOP_TO_BOTTOM,0.009097,0.004935,0.009895,0.016827,-0.004162,0.006932,0.006932
1,00:00:05.782,1,625,7017,476361,14407,42,83,29,60,TOP_TO_BOTTOM,0.004935,0.002724,0.016827,0.030484,-0.002211,0.013657,0.013657
2,00:00:08.951,1,476361,14407,370,10738,29,60,25,28,TOP_TO_BOTTOM,0.002724,0.002064,0.030484,0.054837,-0.000660,0.024353,0.024353
3,00:00:13.151,1,370,10738,476361,14407,25,28,25,51,TOP_TO_BOTTOM,0.002064,0.002211,0.054837,0.046115,0.000147,-0.008722,-0.008722
4,00:00:17.930,1,476361,14407,370,10738,32,50,30,17,TOP_TO_BOTTOM,0.003147,0.002531,0.030491,0.037903,-0.000616,0.007412,0.007412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,01:33:18.517,2,56,2548,365446,12654,46,65,40,89,BOTTOM_TO_TOP,0.007408,0.004269,0.010481,0.020585,-0.003139,0.010104,-0.003139
890,01:33:23.587,2,365446,12654,56,2548,40,89,61,91,BOTTOM_TO_TOP,0.004269,0.013198,0.020585,0.006003,0.008928,-0.014582,0.008928
891,01:33:24.921,2,56,2548,221069,10916,61,91,63,69,BOTTOM_TO_TOP,0.013198,0.019628,0.006003,0.004142,0.006431,-0.001861,0.006431
892,01:33:36.229,2,134102,7863,118,7387,72,30,80,17,BOTTOM_TO_TOP,0.032604,0.046115,0.003274,0.002211,0.013511,-0.001062,0.013511


# Player Locations

In [30]:
columns_to_drop = ['frame_id', 'extrapolated']
existing_columns_to_drop = [col for col in columns_to_drop if col in tracking_df.columns]
tracking_df.drop(columns=existing_columns_to_drop, inplace=True)

In [31]:
if not {'timestamp', 'period'}.issubset(tracking_df.index.names):    
    tracking_df['timestamp'] = tracking_df.apply(standardize_timestamp, axis=1)
    tracking_df.set_index(['timestamp', 'half'], inplace=True)
    tracking_df.rename_axis(index={'timestamp': 'timestamp', 'half': 'period'}, inplace=True)
tracking_df.sample(3)

,,match_id,object_id,x,y,z
timestamp,period,,,,,
00:16:53.600000,1,952209,3837,-5.14,10.11,0.0
00:15:43.900000,1,952209,7017,2.06,-3.70,0.0
00:11:20.300000,1,952209,16320,-20.76,-16.68,0.0


In [32]:
passes['quantizedTimestamp'] = passes['matchTimestamp'].apply(round_to_tenth_of_second)
if not {'timestamp', 'period'}.issubset(passes.index.names):
    passes.set_index(['quantizedTimestamp', 'matchPeriod'], inplace=True)
    passes.rename_axis(index={'quantizedTimestamp': 'timestamp', 'matchPeriod': 'period'}, inplace=True)

In [33]:
joined_df = passes.join(tracking_df, how='inner', validate='one_to_many')
joined_df.drop(columns=['matchTimestamp'], inplace=True)

In [34]:
columns_to_drop = ['match_id']
existing_columns_to_drop = [col for col in columns_to_drop if col in joined_df.columns]
joined_df.drop(columns=existing_columns_to_drop, inplace=True)

joined_df['object_id'] = joined_df['object_id'].astype(int)

In [35]:
columns_to_prefix = ['object_id', 'x', 'y', 'z']
prefix = 'tracking.'
joined_df.rename(columns={col: prefix + col for col in columns_to_prefix}, inplace=True)

In [36]:
joined_df.head(22)

player.id.wyscout  player.id.skillcorner  \
timestamp       period                                             
00:00:01.900000 1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   

                        pass.recipient.id.wyscout  \
timestamp       period                              
00:00:01.900000 1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   

                        pass.recipient.id.skillcorner  location.x  location.y  \
timestamp       period                                                          
00:00:01.900000 1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51          50   
                1                                7017          51

# Normalize Pitch Coordinates

In [38]:
start_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['location.x'], row['location.y'], pitch_length, pitch_width), 
    axis=1)
end_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['pass.endLocation.x'], row['pass.endLocation.y'], pitch_length, pitch_width), 
    axis=1)
joined_df[['location.x', 'location.y']] = start_locations.apply(pd.Series)
joined_df[['pass.endLocation.x', 'pass.endLocation.y']] = end_locations.apply(pd.Series)

# Identify Tracking Objects

In [39]:
def is_self(row):
    return row['player.id.skillcorner'] == row['tracking.object_id']

def is_teammate(row):
    if row['tracking.object_id'] == -1:
        return False
    player_team = player_team_dict[row['player.id.skillcorner']]
    tracking_player_team = player_team_dict[row['tracking.object_id']]
    return player_team == tracking_player_team

In [40]:
joined_df['tracking.is_self'] = joined_df.apply(is_self, axis=1)
joined_df['tracking.is_teammate'] = joined_df.apply(is_teammate, axis=1)

In [41]:
joined_df.sample(5)

,,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,start_xt_right,end_xt_right,start_xt_left,end_xt_left,dxt_right,dxt_left,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.is_self,tracking.is_teammate
timestamp,period,,,,,,,,,,,,,,,,,,,,,,
00:25:19.000000,1,410062,11630,231139,5922,22.235294,28.901053,13.764706,10.993333,BOTTOM_TO_TOP,0.002144,0.004116,0.038622,0.018438,0.001972,-0.020184,0.001972,24466,9.97,-4.16,0.0,False,False
00:30:33.600000,1,3682,946,353833,6028,-19.058824,-4.227692,-20.117647,-9.771111,BOTTOM_TO_TOP,0.027044,0.028362,0.003895,0.003824,0.001318,-0.000071,0.001318,11606,9.99,13.30,0.0,False,True
01:10:05.200000,2,118,7387,38,2106,-18.000000,-31.814737,-14.823529,-23.073684,BOTTOM_TO_TOP,0.017789,0.017198,0.004366,0.005552,-0.000591,0.001186,-0.000591,24466,-2.62,9.14,0.0,False,True
01:20:31.600000,2,491458,24466,38,2106,16.941176,14.660000,6.352941,4.227692,BOTTOM_TO_TOP,0.003350,0.006590,0.021079,0.013966,0.003240,-0.007113,0.003240,7863,-3.75,14.64,0.0,False,True
00:00:52.300000,1,370,10738,625,7017,36.916667,-13.437778,20.117647,-7.046154,TOP_TO_BOTTOM,0.001290,0.002981,0.107793,0.035906,0.001692,-0.071887,-0.071887,12218,11.15,-1.75,0.0,False,False


# Defender Responsibility

In [42]:
joined_df['responsibility'] = joined_df.apply(responsibility, axis=1, pass_length_factor=1)
joined_df['responsibility'] = np.where(joined_df['tracking.is_teammate'], 0, joined_df['responsibility'])

# Coordinate Normalization Evaluation

In [43]:
joined_df[joined_df['tracking.is_self']][['location.x', 'location.y', 'tracking.x', 'tracking.y']]

,,location.x,location.y,tracking.x,tracking.y
timestamp,period,,,,
00:00:01.900000,1,-1.058824,0.000000,-0.70,0.90
00:00:05.800000,1,8.470588,21.616842,9.87,11.72
00:00:09.000000,1,22.235294,7.046154,22.60,7.11
00:00:13.200000,1,26.470588,-14.660000,22.84,-18.80
00:00:17.900000,1,19.058824,0.000000,20.52,-3.75
...,...,...,...,...,...
01:33:18.500000,2,4.235294,10.382222,-2.42,-12.15
01:33:23.600000,2,10.588235,25.987368,-6.34,-28.24
01:33:24.900000,2,-11.647059,27.444211,4.19,-28.35


# Compute ΔxT (TO DO)

To Do:

    1. Flip ΔxT table depending on direction of play (column 'play_direction' in joined_df)
    2. Determine cell in xt_table map by using correct pitch dimensions. Currently we assume pitch dimensions of 100x100.

In [44]:
joined_df.sample(5)

,,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,start_xt_right,end_xt_right,start_xt_left,end_xt_left,dxt_right,dxt_left,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.is_self,tracking.is_teammate,responsibility
timestamp,period,,,,,,,,,,,,,,,,,,,,,,,
00:23:29.200000,1,231139,5922,3682,946,-2.117647,28.172632,6.352941,20.888421,BOTTOM_TO_TOP,0.007804,0.005603,0.009945,0.014488,-0.002201,0.004543,-0.002201,10082,-15.89,20.58,0.0,False,False,0.0
01:29:12.300000,2,118,7387,56,2548,-11.647059,-30.357895,-11.647059,-18.326667,BOTTOM_TO_TOP,0.012719,0.015673,0.006647,0.006417,0.002955,-0.000230,0.002955,8206,-10.21,20.97,0.0,False,False,0.0
00:17:09.200000,1,370,10738,476361,14407,13.764706,-10.993333,13.764706,10.382222,TOP_TO_BOTTOM,0.004111,0.004116,0.026829,0.018775,0.000006,-0.008054,-0.008054,7017,6.32,10.85,0.0,False,True,0.0
00:55:32.100000,2,134496,7604,393247,16320,42.875000,-2.113846,38.750000,14.660000,TOP_TO_BOTTOM,0.001225,0.001132,0.086028,0.051197,-0.000093,-0.034831,-0.034831,14407,0.94,-23.39,0.0,False,False,0.0
00:49:35.200000,2,118,7387,626551,31930,-10.588235,-31.086316,-33.882353,-6.341538,BOTTOM_TO_TOP,0.011725,0.062691,0.007028,0.001832,0.050966,-0.005196,0.050966,24466,-8.54,0.46,0.0,False,True,0.0


In wyscout coordinates are normalized to the range (0,100) in both spatial dimensions.

In [45]:
cols_to_drop_final = ['start_xt_right', 'start_xt_left', 'end_xt_right', 'end_xt_left', 'dxt_right', 'dxt_left']
joined_df = joined_df.drop(columns=cols_to_drop_final)
joined_df.head(22)

player.id.wyscout  player.id.skillcorner  \
timestamp       period                                             
00:00:01.900000 1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   
                1                     118                   7387   

                        pass.recipient.id.wyscout  \
timestamp       period                              
00:00:01.900000 1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   
                1                             625   

                        pass.recipient.id.skillcorner  location.x  location.y  \
timestamp       period                                                          
00:00:01.900000 1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824         0.0   
                1                                7017   -1.058824